In [66]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
import random

In [67]:
torch.manual_seed(32)
np.random.seed(32)
random.seed(32)
class importing:
    def import_files(file_path):
        try:
            if '.csv' in file_path:
                file = pd.read_csv(file_path)
            else:
                file = pd.read_excel(file_path)
        except Exception as e: 
            print(e)
            print('There is something wrong in importing the file') 
        return file

class firewall(importing):

    def __init__(self,name):
        self.name=name
        self.data_path='Dataset.csv'
        
    def Prepossing(self):
        
        self.data=importing.import_files(self.data_path)
        self.data=self.data[(self.data['Action'] == 'allow')|(self.data['Action']== 'deny')]
        self.X = self.data.drop(columns=['Action'])
        self.y = self.data['Action'].map({'allow': 0, 'deny': 1})
        print('Dataset Prepossing is Done!!')
        
        
    def Iso_for(self):



        isolation_forest = IsolationForest(contamination=0.2847, random_state=42)
        isolation_forest.fit(self.X)
        

        # Predict anomalies in the test set
        pred = isolation_forest.predict(self.X)
        
        # Convert predictions to binary labels (0 for normal, 1 for anomaly)
        pred_final = [1 if i == -1 else 0 for i in pred]

        # Assuming 'Action' column contains the true labels
        y_true_label=self.data['Action']


        y_true=[1 if i == 'deny' else 0 for i in pred]
        # Calculate metrics

        self.train_f1 = f1_score(y_true, pred_final,average='weighted')
        self.train_precision = precision_score(y_true, pred_final,average='weighted')
        self.train_recall = recall_score(y_true, pred_final,average='weighted')
        self.accuracy_iso =accuracy_score(y_true, pred_final)
        
        
    def GNN_model(self):

        #self.Prepossing()
        class GNNModel(nn.Module):
            def __init__(self, input_dim, hidden_dim, output_dim):
                super(GNNModel, self).__init__()
                self.fc1 = nn.Linear(input_dim, hidden_dim)
                self.fc2 = nn.Linear(hidden_dim, output_dim)
                self.relu = nn.ReLU()
                self.softmax = nn.Softmax(dim=1)

            def forward(self, x):
                x = self.fc1(x)
                x = self.relu(x)
                x = self.fc2(x)
                x = self.softmax(x)
                return x

        # Step 4: Train the GNN model
        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(self.data.drop(columns=['Action']), self.data['Action'], test_size=0.2, random_state=42)

        # Convert data to PyTorch tensors
        X_train_tensor = torch.FloatTensor(X_train.values)
        y_train_tensor = torch.LongTensor(y_train.map({'allow': 0, 'deny': 1}).values)
        X_test_tensor = torch.FloatTensor(X_test.values)
        y_test_tensor = torch.LongTensor(y_test.map({'allow': 0, 'deny': 1}).values)

        # Create a custom dataset
        class CustomDataset(Dataset):
            def __init__(self, features, labels):
                self.features = features
                self.labels = labels

            def __len__(self):
                return len(self.features)

            def __getitem__(self, idx):
                return self.features[idx], self.labels[idx]

        train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
        test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

        # Define DataLoader
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=32)

        # Initialize the model
        input_dim = X_train_tensor.shape[1]
        hidden_dim = 64
        output_dim = 2  # Number of classes
        model = GNNModel(input_dim, hidden_dim, output_dim)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # Training loop
        num_epochs = 10
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
    #         print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}')

        # Step 5: Evaluate the model using evaluation metrics
        model.eval()
        predictions = []
        true_labels = []
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                predictions.extend(predicted.tolist())
                true_labels.extend(labels.tolist())

        # Calculate evaluation metrics
        self.f1_GNN = f1_score(true_labels, predictions)
        self.recall_GNN = recall_score(true_labels, predictions)
        self.precision_GNN= precision_score(true_labels, predictions)
        self.accuracy_GNN= accuracy_score(true_labels, predictions)






        
        
    
    def Gradient_boosting(self):
        
        self.Prepossing()
        
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)

        gradient_boosting = GradientBoostingClassifier(n_estimators=100, random_state=42)
        gradient_boosting.fit(X_train, y_train)

        y_train_pred = cross_val_predict(gradient_boosting, X_train, y_train, cv=5)

        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_precision = precision_score(y_train, y_train_pred)
        train_recall = recall_score(y_train, y_train_pred)
        train_f1 = f1_score(y_train, y_train_pred)

        self.train_accuracy_grad = train_accuracy
        self.train_precision_grad = train_precision
        self.train_recall_grad = train_recall
        self.train_f1_grad = train_f1

    

        
    def Results(self):
            # Save results to CSV
        results = {
            "Metric": ["F1 Score", "Precision", "Recall", 'Accuracy'],
            "Isolation_forest": [self.train_f1, self.train_precision, self.train_recall,self.accuracy_iso],
            "Gradient_Boosting": [self.train_f1_grad, self.train_precision_grad, self.train_recall_grad,self.train_accuracy_grad],
            "GNN_Model": [self.f1_GNN, self.precision_GNN, self.recall_GNN,self.accuracy_GNN]
        }

        results_df = pd.DataFrame(results)
        results_df.to_csv("result.csv", index=False)
        print(results_df)    
        
        
def firewall_check():
    
    name='Detection'
    anomalies=firewall(name=name)
    print('Job Started!!')
    anomalies.Prepossing()
    anomalies.Iso_for()
    anomalies.GNN_model()
    anomalies.Gradient_boosting()
    anomalies.Results()
    print('Job finished!!')

In [68]:
firewall_check()

Job Started!!
Dataset Prepossing is Done!!


C:\Users\sayan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dataset Prepossing is Done!!
      Metric  Isolation_forest  Gradient_Boosting  GNN_Model
0   F1 Score          0.834022           0.999163   0.971198
1  Precision          1.000000           0.998996   0.994138
2     Recall          0.715298           0.999331   0.949292
3   Accuracy          0.715298           0.999525   0.983755
Job finished!!
